### Simple Baseline(fcn): 0.61
### CNN: 0.66
### batch_size=256,epoch=200的CNN: 0.73
### vae: 烂中爆烂
### resnet: 烂
### CNN+BN+lr_scheduler:0.76过Medium Baseline (loss能降到2.5e-3,目标1.6e-3)
### 迷你CNN:0.78

In [ ]:
# Training progress bar
!pip install -q qqdm
!pip install torchvision

In [ ]:
import numpy as np
import random
import torch

from torch.utils.data import DataLoader
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import torchvision.transforms as transforms

from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models

from torch.optim import Adam, AdamW

from sklearn.cluster import MiniBatchKMeans
from scipy.cluster.vq import vq, kmeans

from qqdm import qqdm, format_str
import pandas as pd

import pdb  # use pdb.set_trace() to set breakpoints for debugging


In [ ]:

train = np.load('../input/hw8anomaly-detection/data-bin/trainingset.npy', allow_pickle=True)
test = np.load('../input/hw8anomaly-detection/data-bin/testingset.npy', allow_pickle=True)

print(train.shape)
print(test.shape)

In [ ]:
def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(19530615)

# 各种模型

In [ ]:
# maybe it can be smaller
class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(),
#             nn.Conv2d(24, 48, 4, stride=2, padding=1), 
#             nn.BatchNorm2d(48),
#             nn.ReLU(),
#             nn.Conv2d(48, 96, 4, stride=2, padding=1),   # medium: remove this layer
#             nn.ReLU(),
        )
        self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1), # medium: remove this layer
#             nn.ReLU(),
#             nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), 
#             nn.BatchNorm2d(24),
#             nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.BatchNorm2d(3),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

## 数据集归一化

In [ ]:
import torchvision
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, train = True):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)
        if train:
            self.transform = transforms.Compose([
                                transforms.Lambda(lambda x: x.to(torch.float32)),
                                transforms.Lambda(lambda x: 2. * x/255. - 1.),
                                torchvision.transforms.RandomResizedCrop((64, 64), scale=(0.3, 1.0), ratio=(0.75, 1.333)),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation(30, expand=False, center=None),
                                # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
                                ])
        else:
            self.transform = transforms.Compose([
                    transforms.Lambda(lambda x: x.to(torch.float32)),
                    transforms.Lambda(lambda x: 2. * x/255. - 1.),
                    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
                    ])
    def __getitem__(self, index):
        x = self.tensors[index]
        
        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

In [ ]:
!pip install transformers==4.5.0

## 训练参数

In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import LambdaLR
import transformers
# Training hyperparameters
num_epochs = 200
batch_size = 512 # medium: smaller batchsize
learning_rate = 3e-4

# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Model
model_type = 'cnn'   # selecting a model type from {'cnn', 'fcn', 'vae', 'resnet'}
model_classes = {'cnn':conv_autoencoder()}
model = model_classes[model_type].cuda()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate)
optimizer_neg = torch.optim.Adam(
    model.parameters(), lr=learning_rate)
# lr_scheduler
total_steps = train.shape[0] * num_epochs // batch_size
print('总步数: ', total_steps)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=total_steps//5, 
                                            num_training_steps=total_steps) 
scheduler_neg = transformers.get_cosine_schedule_with_warmup(optimizer_neg, num_warmup_steps=total_steps//5,
                                            num_training_steps=total_steps)

In [ ]:
best_loss = np.inf
model.train()

qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
for epoch in qqdm_train:
    tot_loss = list()
    for data in train_dataloader:

        # ===================loading=====================
        if model_type in ['cnn', 'vae', 'resnet']:
            img = data.float().cuda()
        elif model_type in ['fcn']:
            img = data.float().cuda()
            img = img.view(img.shape[0], -1)

        # ===================forward=====================
        output = model(img)
        if model_type in ['vae']:
            loss = loss_vae(output[0], img, output[1], output[2], criterion)
        else:
            loss = criterion(output, img)

        tot_loss.append(loss.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # lr_scheduler
        scheduler.step()
        
#         # 负样本训练
#         tot_loss_neg = list()
#         neg_img = 2*torch.rand(size=img.shape)-1        # torch.rand在[0, 1]之间随机
#         neg_output = model(neg_img.cuda())
#         neg_loss = -criterion(neg_output, neg_img.cuda())      # 差的越远越好
#         tot_loss_neg.append(neg_loss.item())
#         optimizer_neg.zero_grad()
#         neg_loss.backward()
#         optimizer_neg.step()
#         scheduler_neg.step()
#     mean_loss_neg = np.mean(tot_loss_neg)
    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
    # ===================log========================
    qqdm_train.set_infos({
      'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
      'loss': f'{mean_loss:.4f}',
    })
    # ===================save_last========================
    torch.save(model, 'last_model_{}.pt'.format(model_type))

## 输出参数

In [ ]:
eval_batch_size = 256

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data, train=False)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=1)
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = 'best_model_{}.pt'.format(model_type)
model = torch.load(checkpoint_path)
model.eval()

# prediction file 
out_file = 'PREDICTION_FILE.csv'

In [ ]:
anomality = list()
with torch.no_grad():
    for i, data in enumerate(test_dataloader): 
        if model_type in ['cnn', 'vae', 'resnet']:
            img = data.float().cuda()
        elif model_type in ['fcn']:
            img = data.float().cuda()
            img = img.view(img.shape[0], -1)
        else:
            img = data[0].cuda()
        output = model(img)
        if model_type in ['cnn', 'resnet', 'fcn']:
            output = output
        elif model_type in ['res_vae']:
            output = output[0]
        elif model_type in ['vae']: # , 'vqvae'
            output = output[0]
        if model_type in ['fcn']:
            loss = eval_loss(output, img).sum(-1)
        else:
            loss = eval_loss(output, img).sum([1, 2, 3])
        anomality.append(loss)
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()

df = pd.DataFrame(anomality, columns=['Predicted'])
df.to_csv(out_file, index_label = 'Id')